In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../src")
from backtesting import Backtesting

In [ ]:
tsla = pd.read_csv("../data/processed/tsla_clean.csv", index_col="Date", parse_dates=True)
spy = pd.read_csv("../data/processed/spy_clean.csv", index_col="Date", parse_dates=True)
bnd = pd.read_csv("../data/processed/bnd_clean.csv", index_col="Date", parse_dates=True)

In [ ]:
prices = pd.concat([tsla["Adj Close"], spy["Adj Close"], bnd["Adj Close"]], axis=1)
prices.columns = ["TSLA", "SPY", "BND"]

### Define backtesting period

In [ ]:
start_date = "2025-01-01"
end_date = "2026-01-01"
prices_bt = prices.loc[start_date:end_date]

In [ ]:
returns_bt = prices_bt.pct_change().dropna()

Load optimized portfolio weights

In [ ]:
portfolio_weights = pd.read_csv("../data/processed/tsla_optimized_portfolio.csv", index_col="Asset")
weights_strategy = portfolio_weights["Max Sharpe Ratio Weights"]

Define benchmark portfolio

In [ ]:
benchmark_weights = pd.Series([0.0, 0.6, 0.4], index=["TSLA", "SPY", "BND"])  # 60% SPY, 40% BND

### Compute daily portfolio returns

In [ ]:
returns_strategy = (returns_bt * weights_strategy).sum(axis=1)
returns_benchmark = (returns_bt * benchmark_weights).sum(axis=1)

### Compute cumulative returns

In [ ]:
cumulative_strategy = (1 + returns_strategy).cumprod()
cumulative_benchmark = (1 + returns_benchmark).cumprod()

### Compute performance metrics

In [ ]:
bt = Backtesting()

In [ ]:
portfolios = {
    "Strategy (Max Sharpe)": returns_strategy,
    "Benchmark (60/40 SPY/BND)": returns_benchmark
}

# Compute performance metrics
metrics_df = bt.compare_portfolios(portfolios)

# Display
print("Performance Metrics:\n")
print(metrics_df)

Cumulative returns Visualizations

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(cumulative_strategy, label="Strategy (Max Sharpe)")
plt.plot(cumulative_benchmark, label="Benchmark (60/40 SPY/BND)")
plt.title("Cumulative Returns: Strategy vs Benchmark")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Backtest Conclusion

The backtest shows that the Max Sharpe strategy portfolio performed slightly below the benchmark in terms of total return (15.6% vs. 15.9%), but it delivered better risk-adjusted performance:

- Sharpe Ratio: 1.47 vs. 1.36

-  Drawdown: -9.2% vs. -11.3%

This indicates that the strategy achieved higher returns per unit of risk and experienced smaller losses during market drawdowns, which is especially valuable for risk-conscious investors.

Overall, the backtest suggests that a forecast-informed, optimized portfolio can improve risk-adjusted outcomes, even if total returns are similar to a simple passive benchmark.

Limitations: This backtest assumes perfect execution with no transaction costs, uses only one year of out-of-sample data, and relies on the accuracy of the Tesla forecast. Real-world performance may vary, and further testing over multiple periods is recommended.